 <h1 color='black'>Data Preprocessing<h1>

In [ ]:
# importing required libraries
import pandas as pd
import numpy as np

import time
import math
import warnings
warnings.filterwarnings('ignore')

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import (StackingClassifier, VotingClassifier, RandomForestClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

In [9]:
# importing dataset
data = pd.read_csv("KDDTrain+.txt")
data

,0,tcp,ftp_data,SF,491,0.1,0.2,0.3,0.4,0.5,...,0.17,0.03,0.17.1,0.00.6,0.00.7,0.00.8,0.05,0.00.9,normal,20
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20
125968,8,udp,private,SF,105,145,0,0,0,0,...,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal,21
125969,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal,18
125970,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20


In [10]:
# dataset doesn't have column names, so we have to provide it
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty_level"]

data.columns = col_names
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty_level
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune,21


<h1 color='black'>Data Normalization<h1>

In [11]:
# selecting numeric attributes columns from data
numeric_col = data.select_dtypes(include='number').columns
numeric_col

Index(['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
       'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'difficulty_level'],
      dtype='object')

In [12]:
# using standard scaler for normalizing

std_scaler = StandardScaler()
def normalization(df,col):
  for i in col:
    arr = df[i]
    arr = np.array(arr)
    df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
  return df

In [13]:
# data before normalization
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty_level
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune,21


In [15]:
# data after normalization
data = normalization(data.copy(),numeric_col)
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty_level
0,-0.11025,udp,other,SF,-0.007737,-0.004919,-0.014089,-0.089487,-0.007736,-0.095076,...,-1.161035,2.736839,2.367729,-0.289105,-0.639535,-0.624874,-0.387635,-0.376389,normal,-1.965547
1,-0.11025,tcp,private,S0,-0.007762,-0.004919,-0.014089,-0.089487,-0.007736,-0.095076,...,-0.938292,-0.174419,-0.480194,-0.289105,1.608750,1.618946,-0.387635,-0.376389,neptune,-0.219968
2,-0.11025,tcp,http,SF,-0.007723,-0.002891,-0.014089,-0.089487,-0.007736,-0.095076,...,1.066393,-0.439079,-0.383106,0.066249,-0.572087,-0.602436,-0.387635,-0.345086,normal,0.652822
3,-0.11025,tcp,http,SF,-0.007728,-0.004814,-0.014089,-0.089487,-0.007736,-0.095076,...,1.066393,-0.439079,-0.480194,-0.289105,-0.639535,-0.624874,-0.387635,-0.376389,normal,0.652822
4,-0.11025,tcp,private,REJ,-0.007762,-0.004919,-0.014089,-0.089487,-0.007736,-0.095076,...,-1.005115,-0.068555,-0.480194,-0.289105,-0.639535,-0.624874,2.874397,2.753901,neptune,0.652822


<h1 color='black'>One-hot-encoding<h1>

In [18]:

df = data.copy()

le = LabelEncoder()

df['protocol_type'] = le.fit_transform(df['protocol_type'])

df['service'] = le.fit_transform(df['service'])

df['flag'] = le.fit_transform(df['flag'])

df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty_level
0,-0.11025,2,44,9,-0.007737,-0.004919,-0.014089,-0.089487,-0.007736,-0.095076,...,-1.161035,2.736839,2.367729,-0.289105,-0.639535,-0.624874,-0.387635,-0.376389,normal,-1.965547
1,-0.11025,1,49,5,-0.007762,-0.004919,-0.014089,-0.089487,-0.007736,-0.095076,...,-0.938292,-0.174419,-0.480194,-0.289105,1.608750,1.618946,-0.387635,-0.376389,neptune,-0.219968
2,-0.11025,1,24,9,-0.007723,-0.002891,-0.014089,-0.089487,-0.007736,-0.095076,...,1.066393,-0.439079,-0.383106,0.066249,-0.572087,-0.602436,-0.387635,-0.345086,normal,0.652822
3,-0.11025,1,24,9,-0.007728,-0.004814,-0.014089,-0.089487,-0.007736,-0.095076,...,1.066393,-0.439079,-0.480194,-0.289105,-0.639535,-0.624874,-0.387635,-0.376389,normal,0.652822
4,-0.11025,1,49,1,-0.007762,-0.004919,-0.014089,-0.089487,-0.007736,-0.095076,...,-1.005115,-0.068555,-0.480194,-0.289105,-0.639535,-0.624874,2.874397,2.753901,neptune,0.652822


<h1 color='black'>Binary Classification<h1>

In [20]:
label = []
def changeLabel_train(df):
    for i in df.label:
        if i=='normal':
            label.append(1)
        else:
            label.append(0)
            
changeLabel_train(df)

df['label'] = label

In [24]:
y = df['label'].ravel()


In [25]:
df.drop(['label'], axis=1, inplace=True)

<h1 color='black'>Train-Test Split<h1>

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, train_size=0.75)

results = pd.DataFrame(columns = ['S.no', 'Model', 'Execution Time', 'Accuracy', 'F1-score', 'Precision', 
                                  'Recall', 'MCC', 'ROC', 'Kappa', 'Mean Absolute Error', 'Root Mean Squared Error', 'Relative Absolute Error', 'Root Relative Squared Error'])
num = 0

<h1 color='black'>Algorithm Report <h1>

In [36]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, classification_report, matthews_corrcoef, roc_auc_score, cohen_kappa_score, precision_score, mean_absolute_error, mean_squared_error 

def get_report(actual, pred, model, t):
    global num, results
    print("Confusion matrix: \n", confusion_matrix(actual, pred))
    accuracy = accuracy_score(actual, pred)
    print("Accuracy: ", accuracy)
    fscore = f1_score(actual, pred)
    print("F1 Score: ", fscore)
    precision = precision_score(actual, pred)
    print("Precision Score: ", precision)
    recall = recall_score(actual, pred)
    print("Recall Score: ", recall)
    mcc = matthews_corrcoef(actual, pred)
    print("MCC: ", mcc)
    roc = roc_auc_score(actual, pred, multi_class='ovr')
    print("ROC: ", roc)
    kappa = cohen_kappa_score(actual, pred)
    print("Kappa: ", kappa)
    mae = mean_absolute_error(actual, pred)
    print("Mean absolute error: ", mae)
    rmse = mean_squared_error(actual, pred, squared=False)
    print("Root Mean Squared Error: ", rmse)
    rae = sum(np.absolute(np.array(pred) - np.array(actual))) / sum(np.absolute(np.array(actual) - np.mean(np.array(y_train))))
    print("Relative Absolute Error: ", rae)
    rrse = math.sqrt(sum((np.array(actual) - np.array(pred))**2) / sum((np.array(actual) - np.mean(np.array(actual)))**2))
    print("Root Relative Squared Error: ", rrse)
    print("\n")
    print("="*60)
    print("\n")
    num += 1
    results = results.append({'S.no': num, 'Model': model, 'Execution Time': t, 
                              'Accuracy': accuracy, 'F1-score': fscore, 'Precision': precision, 
                              'Recall': recall, 'MCC': mcc, 'ROC': roc, 'Kappa': kappa, 
                              'Mean Absolute Error': mae, 'Root Mean Squared Error': rmse, 
                              'Root Absolute Error': rae, 'Root Relative Squared Error': rrse}, ignore_index=True)

In [37]:
models = [ 
    ('Naive Bayes', BernoulliNB()),
    ('KNN', KNeighborsClassifier(n_neighbors=9, weights = "distance")),
    ('Decision Tree', tree.DecisionTreeClassifier(random_state = 0, max_depth = 8, criterion = "entropy")),
    ('Logistic Regression', LogisticRegression(C = 0.001, max_iter=1000)),
    ('Random Forest', RandomForestClassifier(n_jobs = -1, n_estimators = 500, warm_start = True, max_depth = 6, min_samples_leaf = 2, max_features = 'sqrt', verbose = 0))
]

In [39]:
def predict_values(model, clf):
    print('\033[1m' + model + '\033[0m')
    start = time.process_time()
    clf.fit(X_train, y_train)
    t = time.process_time() - start
    print("Execution time: ", t)
    predicted = clf.predict(X_test)
    get_report(y_test, predicted, model, t)

In [40]:
for model, clf in models:
    predict_values(model, clf)

Naive Bayes
Execution time:  0.125
Confusion matrix: 
 [[12479  2120]
 [  704 16190]]
Accuracy:  0.9103292795224336
F1 Score:  0.9197818429723895
Precision Score:  0.884216275259421
Recall Score:  0.9583284006156032
MCC:  0.8219159193611509
ROC:  0.9065564874507566
Kappa:  0.8185053722114589
Mean absolute error:  0.08967072047756644
Root Mean Squared Error:  0.2994506979079635
Relative Absolute Error:  0.18023351511777055
Root Relative Squared Error:  0.6004979976198974




KNN
Execution time:  0.125
Confusion matrix: 
 [[14578    21]
 [   15 16879]]
Accuracy:  0.998856888832439
F1 Score:  0.9989347221400249
Precision Score:  0.9987573964497042
Recall Score:  0.9991121108085711
MCC:  0.997701580851851
ROC:  0.9988368280599468
Kappa:  0.9977015076445649
Mean absolute error:  0.0011431111675610454
Root Mean Squared Error:  0.033809927056428934
Relative Absolute Error:  0.0022975943853540156
Root Relative Squared Error:  0.06780012081755257




Decision Tree
Execution time:  0.265625
Conf

In [46]:
results.to_csv('results2.csv', index=False)